In [1]:
!pip install -U sentence-transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 3.9 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel

In [20]:
model_name = 'cointegrated/rubert-tiny2'

text_pairs = [
    ("Какой сегодня день?", "Сегодня понедельник."),
    ("Что такое машинное обучение?", "Машинное обучение — это область искусственного интеллекта."),
    ("Как приготовить борщ?", "Для приготовления борща нужны свекла, капуста и мясо."),
    ("Кто написал 'Войну и мир'?", "Роман 'Война и мир' написал Лев Толстой."),
    ("Что такое квантовая механика?", "Квантовая механика изучает поведение микрочастиц."),
]

queries = [p[0] for p in text_pairs]
answers = [p[1] for p in text_pairs]

sen_model = SentenceTransformer(model_name, device=torch.device('cuda:0'))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
embeddings_queries = sen_model.encode(queries)
embeddings_answers = sen_model.encode(answers)

In [22]:
sims = embeddings_queries @ embeddings_answers.T

### Ручками

In [23]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

In [24]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings.cpu().numpy()

In [25]:
embeddings_queries = embed_bert_cls(queries, model, tokenizer)
embeddings_answers = embed_bert_cls(answers, model, tokenizer)

In [26]:
sims = embeddings_queries @ embeddings_answers.T

### getting scores

In [47]:
#default labels (max scored)
def get_best(sims, qs, ans):
  best_match = np.argmax(sims, axis=-1)
  preds = {qs[i] : ans[best_match[i]] for i in range(len(best_match))}
  return preds

#get k best
k = 2
def get_k_best(k, sims, qs, ans):
  best_matches = np.argsort(sims, axis=-1, )
  preds = {qs[i] : np.array(ans)[best_matches[i][:-(k+1)]] for i in range(len(best_matches))}
  return preds

In [50]:
preds = get_best(sims, queries, answers)
preds = get_k_best(2, sims, queries, answers)